In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import os
pd.options.mode.chained_assignment = None

In [2]:
def load_all_paths_parquet_with_text(country, year):
    path = '../data_with_text/' + country + '/' + year + '/*.parquet'
    return sorted(glob.glob(path))

def load_all_paths_parquet_no_text(country, year):
    path = '../data_no_text/' + country + '/' + year + '/*.parquet'
    return sorted(glob.glob(path))

def load_file(path):
    table = pq.read_table(path)
    return table.to_pandas()

def load_files(country, year, number):
    paths = load_all_paths_parquet_no_text(country, year)[:number]
    myDataFrame = pd.DataFrame()
    for path in paths:
        table = pq.read_table(path).to_pandas()
        myDataFrame = myDataFrame.append(table)
    return myDataFrame

In [5]:
df = pd.read_pickle('UK_Oxford_2012.pkl')
df

,JobID,SICCode,JobDate,CanonCity,CanonCounty,LocalAuthorityDistrict,CanonSkillClusters,SOCCode,CanonSkills,date
61,25380327,82.20,2012-01-01,BANBURY,Oxfordshire,Cherwell,Specialised Skills|Sales: General Sales;Specia...,7113,"[Basic Internet Skills, Sales, Telemarketing, ...",2012-01-01
86,25380399,85.42,2012-01-01,OXFORD,Oxfordshire,Oxford,Health Care: Gastroenterology;Specialised Skil...,2211,"[Endoscopy, Gastroenterology]",2012-01-01
139,25380453,86,2012-01-01,OXFORD,Oxfordshire,Oxford,Health Care: Clinical Research;Specialised Ski...,2134,"[Clinical Development, Communication Skills, C...",2012-01-01
462,25398820,None,2012-01-03,OXFORD,Oxfordshire,Oxford,Specialised Skills|Specialised Skills|Environm...,4123,"[Communication Skills, Computer Literacy, Envi...",2012-01-03
463,25398821,None,2012-01-03,OXFORD,Oxfordshire,Oxford,Specialised Skills|Engineering: Electronic Har...,2124,"[Basic Internet Skills, Circuit Board, Circuit...",2012-01-03
...,...,...,...,...,...,...,...,...,...,...
13145,19802577,None,2012-12-30,OXFORD,Oxfordshire,Oxford,Health Care: Basic Living Activities Support;S...,6145,"[Companionship, Dementia knowledge, Meal Prepa...",2012-12-30
13168,19802863,None,2012-12-30,OXFORD,Oxfordshire,Oxford,Health Care: Mental Health Diseases and Disord...,6145,[Dementia knowledge],2012-12-30
13203,19803358,86,2012-12-30,OXFORD,Oxfordshire,Oxford,Health Care: Mental Health Diseases and Disord...,2231,"[Alzheimer's Disease Knowledge, Budgeting, Dem...",2012-12-30
13225,19803480,63.12,2012-12-30,OXFORD,Oxfordshire,Oxford,Education and Training: Teaching;Specialised S...,None,"[Graduate Teaching, Lecturer, Research, Teaching]",2012-12-30


In [6]:

def element_to_list(element):
    return str(element).replace(";", "&").replace('"', "").split("&")

def expand_skill(df):
    df = df[['JobID', 'SICCode','JobDate','CanonCity','CanonCounty',
             'LocalAuthorityDistrict',
             'CanonSkillClusters', 'SOCCode', 'CanonSkills']]
    df['date'] = pd.to_datetime(df['JobDate'])
    
    df['CanonSkillClusters'] = df['CanonSkillClusters'].apply(element_to_list)
    
    s = df.apply(lambda x: pd.Series(x['CanonSkillClusters']), axis=1).stack(
    ).reset_index(level=1, drop=True) # takes a lot of time 
    s.name = 'CanonSkillClusters'
    s = df.drop('CanonSkillClusters', axis=1).join(s)
   
    return s


In [7]:
df = expand_skill(df) ##this takes a lot of time

In [8]:
df['CanonSkillClusters'].iloc[0]

'Business: Business Strategy'

In [ ]:
## if you want to pritn the columns
# for col in df.columns: 
#     print(col) 

In [9]:
df

,JobID,SICCode,JobDate,CanonCity,CanonCounty,LocalAuthorityDistrict,SOCCode,CanonSkills,date,CanonSkillClusters
1,26058507,None,2012-01-29,SALFORD,Oxfordshire,West Oxfordshire,7220,[Strategic Development],2012-01-29,Business: Business Strategy
1,26058507,None,2012-01-29,SALFORD,Oxfordshire,West Oxfordshire,7220,[Strategic Development],2012-01-29,Specialised Skills
2,27150342,221210,2012-03-16,WALLINGFORD,Oxfordshire,South Oxfordshire,2123,"[Communication Skills, Management Training, Me...",2012-03-16,Specialised Skills|Human Resources: Employee T...
2,27150342,221210,2012-03-16,WALLINGFORD,Oxfordshire,South Oxfordshire,2123,"[Communication Skills, Management Training, Me...",2012-03-16,Specialised Skills|Specialised Skills|Speciali...
2,27150342,221210,2012-03-16,WALLINGFORD,Oxfordshire,South Oxfordshire,2123,"[Communication Skills, Management Training, Me...",2012-03-16,Specialised Skills|Specialised Skills
...,...,...,...,...,...,...,...,...,...,...
155358,32145590,86.90,2012-07-01,OXFORD,Oxfordshire,Oxford,2219,"[Audiology, Retail Industry Knowledge]",2012-07-01,Specialised Skills|Specialised Skills
155368,32145635,88.99,2012-07-01,DORCHESTER,Oxfordshire,South Oxfordshire,6145,[],2012-07-01,None
155410,32145847,81.21,2012-07-01,DORCHESTER,Oxfordshire,South Oxfordshire,9233,[Cleaning],2012-07-01,Specialised Skills
155412,32145852,86,2012-07-01,DORCHESTER,Oxfordshire,South Oxfordshire,3562,[Orthopaedics],2012-07-01,Health Care: Orthopedics


In [ ]:
def select_df(df, start_date, end_date, 
              city = False, sic = False, soc = False, county = False):
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    df = df.loc[mask]
    if city != False:
        df = df[(df['CanonCity'] == city)]
    if sic != False:
        df = df[(df['SICCode'].str[:4] == sic)]
    if soc != False:
        df = df[(df['SOCCode'].str[:3] == soc)]
    if county != False:
        df = df[(df['CanonCounty'] == county)]
        
    return df

### 1) Calculate frequency counts of CanonSkills for a sub-sampe of job ads tagged as 
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'



In [ ]:
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
skill_count_df = pd.DataFrame(s1['CanonSkills'].value_counts())
skill_count_df

### 2) Calculate frequency counts of CanonSkills for a sub-sampe of job ads tagged as 
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'
**for each sic**


In [ ]:
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s2 = s1.groupby(["CanonSkills", "SICCode"]).size()
pd.DataFrame(s2)

### 3) Calculate frequency counts of CanonSkills for a sub-sampe of job ads tagged as 
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'
**for each soc**


In [ ]:
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s2 = s1.groupby(["CanonSkills", "SOCCode"]).size()
pd.DataFrame(s2)

### 4) Calculate frequency counts of CanonSkills for a sub-sampe of job ads tagged as 
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'
**for each LocalAuthorityDistrict**


In [ ]:
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s2 = s1.groupby(["CanonSkills", "LocalAuthorityDistrict"]).size()
pd.DataFrame(s2)

### 5) Calculate frequency counts of CanonSkills for a sub-sampe of job ads tagged as 
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'
**for each CanonCity**



In [ ]:
start_date = '2016-01-01'
end_date = '2016-01-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s2 = s1.groupby(["CanonSkills", "CanonCity"]).size()
pd.DataFrame(s2)